In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict, Literal
from dotenv import load_dotenv
from pydantic import BaseModel, Field
import operator

In [ ]:
load_dotenv()

In [ ]:
generative_llm = ChatOpenAI(model="gpt-4o-mini)
evaluator_llm = ChatOpenAI(model="gpt-4o-mini)
optimizer_llm = ChatOpenAI(model="gpt-4o-mini)

In [ ]:
class TweetEvaluation(BaseModel):
  evaluation: Literal["approved", "needs_improvement"] = Field(..., description="Final evaluation result.")
  feedback: str = Field(..., description="feedback for the tweet.")



In [ ]:
structured_evaluator_llm = evaluator_llm.with_structured_output(TweetEvaluation)

In [ ]:
class TweetState(TypedDict):

    topic : str
    tweet : str
    evaluation : Literal["approved" , "needs_improvement"]
    iteration : int 
    max_iteration : int

    tweet_history :Annotated[list[str], operator.add]
    feedback_history : Annotated[list[str], operator.add]

    

In [ ]:
def generate_tweet(state: TweetState):

    # prompt
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day english
""")
    ]

    response = generator_llm.invoke(messages).content

    return {"topic":response, "tweet_history" : [response]}

In [ ]:
graph = StateGraph(TweetState)

graph.add_node("generate" , generate_tweet)

graph.add_edge(START , generate_tweet)
graph.add_edge(generate_tweet , END)

workflow = graph.compile()



In [ ]:
workflow.invoke({"topic": "AI in Pakistan"})